# Experiments

### Setup

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(override=True)

True

Here is the RAG Application that we've been working with throughout this course

In [5]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

# TODO: Configure this model!
MODEL_NAME = "gpt-3.5-turbo"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


### Experiment

Here is a code snippet that should look similar to what you see from the starter code!

There are a few important components here.

1. We have defined an Evaluator
2. We pipe our dataset examples (dict) to the shape of input that our function `langsmith_rag` takes (str) using a target function

In [8]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "Rag Application Golden Data Set"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o"
)

View the evaluation results for experiment: 'gpt-4o-870ec07d' at:
https://smith.langchain.com/o/079bb08e-1664-45cb-96ea-d507cd33eaaf/datasets/2038ea5b-0a1d-45c4-a965-3673f4780cf0/compare?selectedSessions=f3b2b682-7567-45a8-a0d7-4eafe7b3b69d




14it [00:23,  1.69s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What is Langchain course,The LangChain course mentioned is about Deep A...,None,Just a course LOL!!,0,1.139448,fffe35fc-dfd2-4db8-840a-f44bf946c06c,102cad05-8212-45eb-b4b4-77be13fb4b90
1,Is LangSmith compatible with third-party libra...,LangSmith integrates seamlessly with LangChain...,None,"Yes, LangSmith is compatible with numerous thi...",1,1.701692,500d4da3-fac7-4291-8ca9-6b618a3f5b3a,82be0c45-4ff8-41d1-bdd8-78e93c705b05
2,How can I analyze performance metrics in LangS...,You can analyze performance metrics in LangSmi...,None,"To analyze performance metrics in LangSmith, y...",1,1.767412,137590e6-5d38-4a09-920f-741ef983213c,783827b2-526a-4e08-94d6-283c7f1ec3c7
3,What is the purpose of LangSmith in AI develop...,LangSmith is a platform for building productio...,None,LangSmith serves as a comprehensive tool for d...,1,1.569953,b57d23d4-2858-4d05-a9d4-28a6ffafa692,3f267594-2466-41f3-bb68-b2740324a0ea
4,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you can in...",None,You can pass metadata with the @traceable deco...,1,1.641344,3a8d882e-5118-45d5-baf8-dd05369c2afa,f7103549-9e81-4315-b683-a48645bfec38
5,What testing capabilities does LangSmith have?,LangSmith allows for running multiple experime...,None,LangSmith offers capabilities for creating dat...,1,1.644880,5dbe5b70-8370-4dda-b6b8-d36ce8d17543,ddbb7558-55c5-49e2-8dc8-1d5993df0f8e
6,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.582102,6a93845f-ad26-4878-a62e-d9bfd2d3e255,b4540719-800d-44c9-b7d6-7f8c82e38146
7,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.368115,70c9c849-87a2-4f51-8f57-a3a310240885,78f09174-2ce7-44f2-ab59-5b18e088a0d2
8,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for monitoring...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.727643,79f822d3-8d61-4ced-be01-8cd05dfbc452,56b26ae2-2e78-45f7-b0bc-d323a96e913e
9,Does LangSmith support offline evaluation?,LangSmith primarily supports online evaluation...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.345885,afa0a059-783e-4fb7-81e5-0311ea8af752,9b563854-da37-4b9e-abe9-c4f4c00725f7


### Modifying your Application

Now, let's change our model to gpt-35-turbo and see how it performs!

Make this change, and then run this code snippet!

In [7]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-fff1eaf5' at:
https://smith.langchain.com/o/079bb08e-1664-45cb-96ea-d507cd33eaaf/datasets/2038ea5b-0a1d-45c4-a965-3673f4780cf0/compare?selectedSessions=f2329a4b-a7fb-4d22-b1d7-cb6538122910




14it [00:25,  1.83s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What is Langchain course,"The LangChain course mentioned is called ""Deep...",None,Just a course LOL!!,0,1.448166,fffe35fc-dfd2-4db8-840a-f44bf946c06c,9901f8e6-b262-470e-a8a2-9ac79f5c1f91
1,Is LangSmith compatible with third-party libra...,LangSmith integrates seamlessly with LangChain...,None,"Yes, LangSmith is compatible with numerous thi...",1,1.543889,500d4da3-fac7-4291-8ca9-6b618a3f5b3a,6488901e-bc88-4daf-b00e-6b1db3fb8d14
2,How can I analyze performance metrics in LangS...,"In LangSmith, after running an experiment, you...",None,"To analyze performance metrics in LangSmith, y...",1,1.972720,137590e6-5d38-4a09-920f-741ef983213c,8cedd81c-799f-42c9-880d-34260056970b
3,What is the purpose of LangSmith in AI develop...,LangSmith is a platform for building productio...,None,LangSmith serves as a comprehensive tool for d...,1,1.302179,b57d23d4-2858-4d05-a9d4-28a6ffafa692,72989772-0999-4bcd-b6b7-16898e0eeb42
4,How do I pass metadata in with @traceable?,You can pass metadata in with @traceable by in...,None,You can pass metadata with the @traceable deco...,1,1.694799,3a8d882e-5118-45d5-baf8-dd05369c2afa,9ece763c-357d-4a12-bea3-32277dd2850d
5,What testing capabilities does LangSmith have?,LangSmith allows users to run multiple experim...,None,LangSmith offers capabilities for creating dat...,1,1.383383,5dbe5b70-8370-4dda-b6b8-d36ce8d17543,4e837e50-6496-4738-bf4f-e7cea49e248e
6,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.502710,6a93845f-ad26-4878-a62e-d9bfd2d3e255,4e1f9d4f-1420-4139-847d-ceb655ef50f1
7,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,1.663707,70c9c849-87a2-4f51-8f57-a3a310240885,1760686c-299c-4ac1-aa25-7ac766118e1e
8,Can LangSmith be used for finetuning and model...,"LangSmith can be used for monitoring, evaluati...",None,"Yes, LangSmith can be used for fine-tuning and...",0,3.750910,79f822d3-8d61-4ced-be01-8cd05dfbc452,6794a46f-7f18-45d0-9945-aaa8c3169a52
9,Does LangSmith support offline evaluation?,LangSmith documentation does not mention suppo...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.765941,afa0a059-783e-4fb7-81e5-0311ea8af752,80b21098-eec7-46f6-bd68-2b39601ba4e6


### Running over Different pieces of Data

##### Dataset Version

You can execute an experiment on a specific version of a dataset in the sdk by using the `as_of` parameter in `list_examples`

Let's try running on just our initial dataset.

In [9]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, as_of="original"),   # We use as_of to specify a version
    evaluators=[is_concise_enough],
    experiment_prefix="initial dataset version"
)

View the evaluation results for experiment: 'initial dataset version-f3f37eb6' at:
https://smith.langchain.com/o/079bb08e-1664-45cb-96ea-d507cd33eaaf/datasets/2038ea5b-0a1d-45c4-a965-3673f4780cf0/compare?selectedSessions=856ae8f9-f48e-46c2-a038-9e67c073d70d




10it [00:18,  1.84s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you can in...",None,You can pass metadata with the @traceable deco...,1,2.139890,3a8d882e-5118-45d5-baf8-dd05369c2afa,8b204a29-bb63-4749-a47c-ed56e9107ccd
1,What testing capabilities does LangSmith have?,LangSmith in LangChain provides testing capabi...,None,LangSmith offers capabilities for creating dat...,1,1.468031,5dbe5b70-8370-4dda-b6b8-d36ce8d17543,c9bdb40b-ab41-43d0-8b4f-54bd7e4aed92
2,What is LangSmith used for in three sentences?,LangSmith is a platform for building productio...,None,LangSmith is a platform designed for the devel...,1,1.442392,6a93845f-ad26-4878-a62e-d9bfd2d3e255,506e7caa-dcb1-4117-8c53-b80d9a2e8f89
3,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,1.720387,70c9c849-87a2-4f51-8f57-a3a310240885,2cee25f0-927c-4b6a-8e0e-15f8c0d12bcc
4,Can LangSmith be used for finetuning and model...,"LangSmith is primarily focused on monitoring, ...",None,"Yes, LangSmith can be used for fine-tuning and...",1,1.546410,79f822d3-8d61-4ced-be01-8cd05dfbc452,cca9ac57-4a20-4d50-a840-0d08aad42cd6
5,Does LangSmith support offline evaluation?,"LangSmith focuses on online evaluations, provi...",None,"Yes, LangSmith supports offline evaluation thr...",0,2.205622,afa0a059-783e-4fb7-81e5-0311ea8af752,3c04a71c-2695-4e34-8bff-56d3e06f7ed0
6,Does LangSmith support online evaluation?,LangSmith supports online evaluations for cont...,None,"Yes, LangSmith supports online evaluation as a...",0,2.084273,b2a3844f-14d5-4e15-9733-835112a540cc,7a126782-d95a-463f-83d7-8d0f5f468267
7,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents,...",None,"Yes, LangSmith can be used to evaluate agents....",1,1.402356,b66d96fd-57d4-42d6-965f-38b675beabf9,8715b858-7ed0-4216-9788-66c079341cb9
8,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.716163,c0a4064c-9791-4182-b6e5-b363c759dbfe,9c4c2b0f-aac4-46c9-853e-854619724332
9,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith while using Lan...,0,2.142593,ebf2bb11-d1fb-438d-b53d-01d464f3ce07,b7b188f0-e790-408d-a718-662b4b74eec4


##### Dataset Split

You can run an experiment on a specific split of your dataset, let's try running on the Crucial Examples split.

In [10]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, splits=["Crucial Examples"]),  # We pass in a list of Splits
    evaluators=[is_concise_enough],
    experiment_prefix="Crucial Examples split"
)

View the evaluation results for experiment: 'Crucial Examples split-f670349c' at:
https://smith.langchain.com/o/079bb08e-1664-45cb-96ea-d507cd33eaaf/datasets/2038ea5b-0a1d-45c4-a965-3673f4780cf0/compare?selectedSessions=e1377896-7635-4835-8643-aac15615ad90




6it [00:10,  1.70s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Is LangSmith compatible with third-party libra...,LangSmith integrates seamlessly with LangChain...,None,"Yes, LangSmith is compatible with numerous thi...",1,1.607945,500d4da3-fac7-4291-8ca9-6b618a3f5b3a,8b83d6fa-516f-4964-934d-6ae3edad0133
1,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.585465,6a93845f-ad26-4878-a62e-d9bfd2d3e255,40548f6e-24e8-4daf-8d4f-c942d8117622
2,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for monitoring...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.724165,79f822d3-8d61-4ced-be01-8cd05dfbc452,e605798e-51dc-4678-8dab-193645d1cf9c
3,Does LangSmith support online evaluation?,"Yes, LangSmith does support online evaluation ...",None,"Yes, LangSmith supports online evaluation as a...",1,1.299011,b2a3844f-14d5-4e15-9733-835112a540cc,b0b0d7a3-d36f-44d8-ae12-d3b03a815f17
4,What is the purpose of LangSmith in AI develop...,LangSmith is a platform for building productio...,None,LangSmith serves as a comprehensive tool for d...,1,2.042069,b57d23d4-2858-4d05-a9d4-28a6ffafa692,791c9f0f-2513-4da4-add1-464d703b78af
5,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.401684,c0a4064c-9791-4182-b6e5-b363c759dbfe,4f25f49f-282d-4b44-a120-6eb91ff41366


##### Specific Data Points

You can specify individual data points to run an experiment over as well

In [12]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # We pass in a specific list of example_ids
            # TODO: You will need to paste in your own example ids for this to work!
            "137590e6-5d38-4a09-920f-741ef983213c",
            "5dbe5b70-8370-4dda-b6b8-d36ce8d17543"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

View the evaluation results for experiment: 'two specific example ids-08d2de47' at:
https://smith.langchain.com/o/079bb08e-1664-45cb-96ea-d507cd33eaaf/datasets/2038ea5b-0a1d-45c4-a965-3673f4780cf0/compare?selectedSessions=4b2a1866-6d3b-48ca-b7b6-b7ff73196438




2it [00:04,  2.14s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How can I analyze performance metrics in LangS...,You can analyze performance metrics in LangSmi...,None,"To analyze performance metrics in LangSmith, y...",1,2.036945,137590e6-5d38-4a09-920f-741ef983213c,091a79a8-be48-45cf-aecf-57cba33a999f
1,What testing capabilities does LangSmith have?,LangSmith offers the ability to run multiple e...,None,LangSmith offers capabilities for creating dat...,1,1.479741,5dbe5b70-8370-4dda-b6b8-d36ce8d17543,d97c1086-9384-45a6-81c9-b51126a26099


### Other Parameters

##### Repetitions

You can run an experiment several times to make sure you have consistent results

In [13]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-02532ad1' at:
https://smith.langchain.com/o/079bb08e-1664-45cb-96ea-d507cd33eaaf/datasets/2038ea5b-0a1d-45c4-a965-3673f4780cf0/compare?selectedSessions=07db753c-d7c3-40c9-befb-470e69b234cd




28it [00:54,  1.95s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Is LangSmith compatible with third-party libra...,"Yes, LangSmith is compatible with third-party ...",None,"Yes, LangSmith is compatible with numerous thi...",1,1.528046,500d4da3-fac7-4291-8ca9-6b618a3f5b3a,d09f876a-36e9-4a2e-9801-fd687428077b
1,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.613141,6a93845f-ad26-4878-a62e-d9bfd2d3e255,d5ed4c6d-251f-4b68-884c-29655ef6d2f0
2,Can LangSmith be used for finetuning and model...,"LangSmith is mainly focused on monitoring, eva...",None,"Yes, LangSmith can be used for fine-tuning and...",1,1.777283,79f822d3-8d61-4ced-be01-8cd05dfbc452,c250ac23-9023-464b-9f8d-0970c65f92a4
3,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation with...",None,"Yes, LangSmith supports online evaluation as a...",0,1.731509,b2a3844f-14d5-4e15-9733-835112a540cc,0063ac1c-4476-4d70-a986-aae86f08f873
4,What is the purpose of LangSmith in AI develop...,LangSmith is a platform for building productio...,None,LangSmith serves as a comprehensive tool for d...,1,1.291417,b57d23d4-2858-4d05-a9d4-28a6ffafa692,baab66fd-e7f9-48b9-aecc-977a4a6015ec
5,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.816778,c0a4064c-9791-4182-b6e5-b363c759dbfe,f2ed9d2a-f1bb-4005-9682-92128484e989
6,What is Langchain course,"The LangChain course mentioned is called ""Deep...",None,Just a course LOL!!,0,1.465680,fffe35fc-dfd2-4db8-840a-f44bf946c06c,b8c72d7a-0344-4951-827e-dd74f40a0cdf
7,How can I analyze performance metrics in LangS...,"To analyze performance metrics in LangSmith, y...",None,"To analyze performance metrics in LangSmith, y...",1,2.056577,137590e6-5d38-4a09-920f-741ef983213c,71746bad-da5a-425d-bf86-10904f0252fc
8,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you can in...",None,You can pass metadata with the @traceable deco...,1,1.696325,3a8d882e-5118-45d5-baf8-dd05369c2afa,ce014dc6-af14-4c64-b24e-b714841635f1
9,What testing capabilities does LangSmith have?,LangSmith in LangChain offers testing capabili...,None,LangSmith offers capabilities for creating dat...,1,1.741342,5dbe5b70-8370-4dda-b6b8-d36ce8d17543,403d9294-1138-47e2-b2f8-117c03d9b085


##### Concurrency
You can also kick off concurrent threads of execution to make your experiments finish faster!

In [14]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-ac383d89' at:
https://smith.langchain.com/o/079bb08e-1664-45cb-96ea-d507cd33eaaf/datasets/2038ea5b-0a1d-45c4-a965-3673f4780cf0/compare?selectedSessions=a4fd7a0b-76c2-4651-84b9-437ead33709f




14it [00:09,  1.51it/s]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Is LangSmith compatible with third-party libra...,"Yes, LangSmith is compatible with third-party ...",None,"Yes, LangSmith is compatible with numerous thi...",1,1.327565,500d4da3-fac7-4291-8ca9-6b618a3f5b3a,abfe0821-5cf9-4425-97f6-8d6714f53b68
1,What is LangSmith used for in three sentences?,LangSmith is used as a platform for building p...,None,LangSmith is a platform designed for the devel...,1,2.316165,6a93845f-ad26-4878-a62e-d9bfd2d3e255,31d32ef3-6233-41b8-bd73-6f59a611d0cb
2,Can LangSmith be used for finetuning and model...,"LangSmith is mainly focused on observability, ...",None,"Yes, LangSmith can be used for fine-tuning and...",1,2.630542,79f822d3-8d61-4ced-be01-8cd05dfbc452,506be52a-79ed-4073-8c4b-536b5163ddcd
3,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation thro...",None,"Yes, LangSmith supports online evaluation as a...",0,1.880279,b2a3844f-14d5-4e15-9733-835112a540cc,fa7aa043-a517-4939-851f-71d87f36e04a
4,What is the purpose of LangSmith in AI develop...,LangSmith is a platform for building productio...,None,LangSmith serves as a comprehensive tool for d...,1,1.991707,b57d23d4-2858-4d05-a9d4-28a6ffafa692,2c8ad60a-f374-4cd1-a8e9-4a984fab8fc4
5,What is Langchain course,The Langchain course mentioned in the provided...,None,Just a course LOL!!,0,1.169806,fffe35fc-dfd2-4db8-840a-f44bf946c06c,1430f11a-c1b4-468c-8b79-6aee227b604d
6,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you fi...",None,To trace with the @traceable decorator in Pyth...,1,1.753903,c0a4064c-9791-4182-b6e5-b363c759dbfe,9f385b0e-f57e-4b2c-8e8c-daa9fc2c8588
7,How do I pass metadata in with @traceable?,"To pass metadata in with `@traceable`, you can...",None,You can pass metadata with the @traceable deco...,1,1.479767,3a8d882e-5118-45d5-baf8-dd05369c2afa,0917356c-fc3a-42bf-bc74-589547880b16
8,How can I analyze performance metrics in LangS...,"To analyze performance metrics in LangSmith, y...",None,"To analyze performance metrics in LangSmith, y...",1,1.841019,137590e6-5d38-4a09-920f-741ef983213c,5de7e735-0bfe-4538-8276-8d429d10c09d
9,What testing capabilities does LangSmith have?,LangSmith in LangChain offers testing capabili...,None,LangSmith offers capabilities for creating dat...,1,2.484177,5dbe5b70-8370-4dda-b6b8-d36ce8d17543,55732ca3-7924-4c85-9350-f7392f7c838c


##### Metadata 

You can (and should) add metadata to your experiments, to make them easier to find in the UI

In [15]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={  # We can pass custom metadata for the experiment, such as the model name
        "model_name": MODEL_NAME
    }
)

View the evaluation results for experiment: 'metadata added-51ba9281' at:
https://smith.langchain.com/o/079bb08e-1664-45cb-96ea-d507cd33eaaf/datasets/2038ea5b-0a1d-45c4-a965-3673f4780cf0/compare?selectedSessions=67724049-6e0b-4057-8c51-e4f9a6ec1a24




14it [00:24,  1.72s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Is LangSmith compatible with third-party libra...,"Yes, LangSmith is compatible with third-party ...",None,"Yes, LangSmith is compatible with numerous thi...",1,1.228154,500d4da3-fac7-4291-8ca9-6b618a3f5b3a,c5520604-4ef7-47b0-9a61-0763911f3c5d
1,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.578744,6a93845f-ad26-4878-a62e-d9bfd2d3e255,6265859f-b773-4c15-89d3-a053b9305f05
2,Can LangSmith be used for finetuning and model...,"LangSmith is primarily focused on monitoring, ...",None,"Yes, LangSmith can be used for fine-tuning and...",1,1.766719,79f822d3-8d61-4ced-be01-8cd05dfbc452,6a4c6e05-fdea-43c7-8e45-ddf69c541ea5
3,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation thro...",None,"Yes, LangSmith supports online evaluation as a...",1,1.380010,b2a3844f-14d5-4e15-9733-835112a540cc,61b69dd6-67c2-4be9-9c0f-cd40e7217b91
4,What is the purpose of LangSmith in AI develop...,LangSmith is a platform for building productio...,None,LangSmith serves as a comprehensive tool for d...,1,1.570446,b57d23d4-2858-4d05-a9d4-28a6ffafa692,534bdeec-2c19-42a8-ac1e-7cefb8e548b3
5,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.479560,c0a4064c-9791-4182-b6e5-b363c759dbfe,54faf9f2-e9a1-443a-a60c-7bfbbf50fe8b
6,What is Langchain course,"The LangChain course is called ""Deep Agents"" a...",None,Just a course LOL!!,0,1.489664,fffe35fc-dfd2-4db8-840a-f44bf946c06c,a2e595d4-d2ad-4872-af14-7a7c25d4d86b
7,How can I analyze performance metrics in LangS...,"To analyze performance metrics in LangSmith, y...",None,"To analyze performance metrics in LangSmith, y...",1,1.698223,137590e6-5d38-4a09-920f-741ef983213c,e7bfce13-b772-4aa0-a340-17a2a7295c9c
8,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you can in...",None,You can pass metadata with the @traceable deco...,1,2.875804,3a8d882e-5118-45d5-baf8-dd05369c2afa,a0427760-aa7e-4a97-8eef-355399115398
9,What testing capabilities does LangSmith have?,LangSmith in LangChain offers testing capabili...,None,LangSmith offers capabilities for creating dat...,1,1.401661,5dbe5b70-8370-4dda-b6b8-d36ce8d17543,275cf290-4b30-4a67-9a67-b2af711d86c4
